In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.impute import SimpleImputer

covariates = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/proteomics/Covariates.csv").iloc[:,1:]

covariate_cols = covariates.columns[1:]




In [ ]:
covariates["Ethnicity"].value_counts()

In [ ]:
plates_proteomics =  pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/proteomics/Plate_proteomics.csv").iloc[:,1:]

In [ ]:
plates_proteomics

In [ ]:
plates_proteomics.rename(columns={"p30901_i0":"plate"},inplace=True)

In [ ]:
covariates_plate = pd.merge(plates_proteomics[["eid","plate"]], covariates, on=["eid"])

In [ ]:
assessment_center = pd.read_csv("p54_Assessment_centre.csv")

In [ ]:
assessment_center.rename(columns={"p54_i0":"assessment_center"},inplace=True)

In [ ]:
assessment_center

In [ ]:
assessment_center[assessment_center["eid"].isin(covariates_plate["eid"].unique().tolist())]

In [ ]:
covariates_plate_assessment = pd.merge(covariates_plate, assessment_center[["eid","assessment_center"]], on=["eid"])

In [ ]:
covariates_plate_assessment

In [ ]:
covariates_plate_assessment

In [ ]:
parkinson_disease = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/processed/diagnoses_extracted/Parkinson.csv")["eid"]

In [ ]:
covariates_plate_assessment["Parkinson_diagnosis"] = covariates_plate_assessment["eid"].isin(parkinson_disease).astype(int)

In [ ]:
covariates_plate_assessment["Parkinson_diagnosis"].value_counts()

In [ ]:
path = "/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/proteomics"

In [ ]:
import pandas as pd

olink_assay_version = pd.read_csv(f'{path}/olink_assay_version.dat', sep='\t')  


In [ ]:
olink_panel_lot = pd.read_csv(f'{path}/olink_panel_lot_number.dat', sep='\t')  

In [ ]:
olink_panel_lot

In [ ]:
olink_batch_number = pd.read_csv(f'{path}/olink_batch_number.dat', sep='\t')  

In [ ]:
olink_batch_number.rename(columns={"PlateID":"plate"},inplace=True)

In [ ]:
covariates_plate_assessment_batch = pd.merge(olink_batch_number,covariates_plate_assessment, on="plate")

In [ ]:
covariates_plate_assessment_batch["Ethnicity"] = covariates_plate_assessment_batch["Ethnicity"].apply(
    lambda x: int(x) if pd.notna(x) else x
)


In [ ]:

covariates_plate_assessment_batch["sample_age"] = covariates_plate_assessment_batch["sample_age"].str.extract(r"(\d+)").astype(float)

ethnicity_map = {
    1: "White", 1001: "White", 1002: "White", 1003: "White",
    2: "Mixed", 2001: "Mixed", 2002: "Mixed", 2003: "Mixed", 2004: "Mixed",
    3: "Asian", 3001: "Asian", 3002: "Asian", 3003: "Asian", 3004: "Asian",
    4: "Black", 4001: "Black", 4002: "Black", 4003: "Black",
    5: "Asian",  
    6: "Other/Unknown",
    -1: "Other/Unknown",
    -3: "Other/Unknown"
}

covariates_plate_assessment_batch['alcohol'] = covariates_plate_assessment_batch['alcohol'].astype('object')
covariates_plate_assessment_batch['smoking']= covariates_plate_assessment_batch['smoking'].astype('object')
covariates_plate_assessment_batch["Ethnicity"] = covariates_plate_assessment_batch["Ethnicity"].map(ethnicity_map)
covariates_plate_assessment_batch["alcohol"] = covariates_plate_assessment_batch["alcohol"].fillna(-3)
covariates_plate_assessment_batch["smoking"] = covariates_plate_assessment_batch["smoking"].fillna(-3)
covariates_plate_assessment_batch["Ethnicity"] = covariates_plate_assessment_batch["Ethnicity"].fillna("Other/Unknown")

In [ ]:
covariates_plate_assessment_batch = covariates_plate_assessment_batch.set_index(["eid"])

In [ ]:
df_all

In [ ]:
protein_metadata = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/proteomics/protein_metadata.csv")

In [ ]:
df_all = pd.merge(covariates_plate_assessment_batch, protein_metadata[["eid","p30900_i0","p30901_i0","p30902_i0"]], on="eid")

In [ ]:
df_all.rename(columns={"p30900_i0":"number_proteins","p30902_i0":"well"},inplace=True)

In [ ]:
(df_all["plate"] == df_all["p30901_i0"]).all()


In [ ]:
df_all.drop(columns=["p30901_i0"],inplace=True)

In [ ]:
df_all["assessment_center"] = df_all["assessment_center"].astype("str")

In [ ]:
df_all["plate"] = df_all["plate"].astype("str")
df_all["Batch"] = df_all["Batch"].astype("str")

In [ ]:
df_all.to_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/proteomics/Processed_all_covariates.csv")

In [ ]:
df_all["Ethnicity"].value_counts()